In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor


In [2]:
#공개 데이터 셋에서 학습 데이터를 내려받습니다.
training_data = datasets.FashionMNIST(
    root = "data",
    train = True,
    download = True,
    transform = ToTensor(),
)

#공개 데이터 셋에서 테스트 데이터를 받습니다. 
test_data = datasets.FashionMNIST(
    root ="data",
    train = False,
    download = True,
    transform = ToTensor(),
)

In [3]:
batch_size = 64
#데이터로더를 생성합니다.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y : {y.shape}, {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y : torch.Size([64]), torch.int64


In [10]:
#학습에 사용할 CPU나 GPU 장치를 얻습니다.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

#모델을 정의합니다.
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [11]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-3)

In [12]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # 예측 오류 계산
        pred = model(X)
        loss = loss_fn(pred, y)

        # 역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch*len(X)
            print(f"loss: {loss :>7} [{current :>5d}/{size :>5d}]")


In [13]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == 1).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Acurracy : {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f}\n")

In [14]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n --------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
 --------------
loss: 2.3013627529144287 [    0/60000]
loss: 2.287651300430298 [ 6400/60000]
loss: 2.265547752380371 [12800/60000]
loss: 2.2629354000091553 [19200/60000]
loss: 2.2518482208251953 [25600/60000]
loss: 2.219064712524414 [32000/60000]
loss: 2.2295873165130615 [38400/60000]
loss: 2.191493034362793 [44800/60000]
loss: 2.1895408630371094 [51200/60000]
loss: 2.1532809734344482 [57600/60000]
Test Error: 
 Acurracy : 8.2%, Avg loss: 2.152911

Epoch 2
 --------------
loss: 2.1660733222961426 [    0/60000]
loss: 2.153653383255005 [ 6400/60000]
loss: 2.0937576293945312 [12800/60000]
loss: 2.1128170490264893 [19200/60000]
loss: 2.0658881664276123 [25600/60000]
loss: 2.00071120262146 [32000/60000]
loss: 2.031933069229126 [38400/60000]
loss: 1.940489649772644 [44800/60000]
loss: 1.9564422369003296 [51200/60000]
loss: 1.8781332969665527 [57600/60000]
Test Error: 
 Acurracy : 12.1%, Avg loss: 1.880773

Epoch 3
 --------------
loss: 1.9163113832473755 [    0/60000]
loss: 1.8849816

In [15]:
torch.save(model.state_dict(), "model.pth")
print("Saved Pytorch Model state to model.pth")

Saved Pytorch Model state to model.pth


In [16]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [17]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred=model(x)
    print(f"pred : {pred}")
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f"Predicted : {predicted}, Actual : {actual}") 

pred : tensor([[-2.0462, -2.4950, -0.6949, -1.8216, -0.8251,  2.3635, -0.9225,  2.6615,
          1.8456,  3.0700]])
Predicted : Ankle boot, Actual : Ankle boot
